# Compare patient with normative samples from other studies

### import modules

In [1]:
import sys
import os
import fnmatch

import numpy as np
import pandas as pd
from scipy import stats

sys.path.append('../modules/')
import case_stats as cs

### Reading the eyes in the mind

In [2]:
group_mean = 28.6
group_std = 3.2
group_n = 50
pat_mean = 26

max_score = 36

percent correct responses

In [3]:
pat_percent = (float(pat_mean) / max_score) * 100
'%0.f' % pat_percent

'72'

### convert each participant's scores into z-scores, then transform the z-scores into percentiles

In [4]:
'z= %.3f ; percentile= %.0f' % cs.make_percentiles(group_mean, group_std,
                                                   pat_mean)

/home/mwegrzyn/Documents/MS_PROSOP/venv/local/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


'z= -0.813 ; percentile= 21'

### compute a t-test patient vs controls, using the fromula by Crawford and Howell

In [5]:
t, df, p = cs.ttest_single(pat_mean, group_mean, group_std, group_n)
print 't=%.2f, df=%.0f p=%.4f' % (t, df, p)

t=-0.80, df=49 p=0.4250


In [6]:
def make_df(group_mean, group_std, group_n, pat_mean, max_score, test_name,
            domain_name):

    # get percent correct
    pat_percent = (float(pat_mean) / max_score) * 100

    # make percentile score
    z, percentile_score = cs.make_percentiles(group_mean, group_std, pat_mean)

    # compute inferential statistics
    t, df, p = cs.ttest_single(pat_mean, group_mean, group_std, group_n)

    # put data into dictionary
    d = {
        '% correct': '%0.f' % pat_percent,
        'z': z,
        'percentile': '%0.f' % percentile_score,
        't': '%.2f' % t,
        'df': int(df),
        'p': '%.3f' % p
    }

    # transform into dataframe
    result_df = pd.DataFrame(d, index=[test_name])
    # format the dataframe
    result_df = result_df.loc[:,
                              ['% correct', 'z', 'percentile', 't', 'df', 'p']]
    result_df.index = [[domain_name] * result_df.shape[0], result_df.index]

    return result_df

### compute all statistics 

In [7]:
group_mean = 28.6
group_std = 3.2
group_n = 50
pat_mean = 26
max_score = 36

eyes_df = make_df(
    group_mean,
    group_std,
    group_n,
    pat_mean,
    max_score,
    test_name='Eyes test',
    domain_name='Face evaluation')

eyes_df

,,% correct,z,percentile,t,df,p
Face evaluation,Eyes test,72,-0.812299,21,-0.80,49,0.425


### Bergen Test

In [8]:
group_mean = 110.24
group_std = 24.67
group_n = 174
pat_mean = 48
max_score = 144

blrda_df = make_df(
    group_mean,
    group_std,
    group_n,
    pat_mean,
    max_score,
    test_name='BRLD-A',
    domain_name='Visuo-spatial')

blrda_df

,,% correct,z,percentile,t,df,p
Visuo-spatial,BRLD-A,33,-2.523836,1,-2.52,173,0.013


In [9]:
group_mean = 109.73
group_std = 27.64
group_n = 174
pat_mean = 54
max_score = 144

blrdb_df = make_df(
    group_mean,
    group_std,
    group_n,
    pat_mean,
    max_score,
    test_name='BRLD-B',
    domain_name='Visuo-spatial')

blrdb_df

,,% correct,z,percentile,t,df,p
Visuo-spatial,BRLD-B,38,-2.01505,2,-2.01,173,0.046


In [10]:
blrd_df = pd.concat([blrda_df, blrdb_df])
blrd_df

% correct         z percentile      t   df      p
Visuo-spatial BRLD-A        33 -2.523836          1  -2.52  173  0.013
              BRLD-B        38 -2.015050          2  -2.01  173  0.046

### LPS 2

In [11]:
group_mean = 76  # iq 100
group_std = (94 - 58) / 2.  # range from iq 85 to iq 115
group_n = 129
pat_1, pat_2, pat_3 = 12, 26, 20
pat_mean = pat_1 + pat_2 + pat_3
max_score = 40 + 40 + 40

lps2_df = make_df(
    group_mean,
    group_std,
    group_n,
    pat_mean,
    max_score,
    test_name='LPS2-Visual',
    domain_name='Visuo-spatial')

lps2_df

,,% correct,z,percentile,t,df,p
Visuo-spatial,LPS2-Visual,48,-0.998214,16,-1.00,128,0.321


### store everything in one table

In [12]:
results_df = pd.concat([eyes_df, blrd_df, lps2_df])
results_df

% correct         z percentile      t   df      p
Face evaluation Eyes test          72 -0.812299         21  -0.80   49  0.425
Visuo-spatial   BRLD-A             33 -2.523836          1  -2.52  173  0.013
                BRLD-B             38 -2.015050          2  -2.01  173  0.046
                LPS2-Visual        48 -0.998214         16  -1.00  128  0.321

In [13]:
results_df.to_csv('../reports/tables/penandpaper_main_results.csv')